## Simple Dask Client Demo

### Simple Code

In [ ]:
from time import sleep

def inc(x):
    sleep(1)
    return x + 1

def add(x, y):
    sleep(1)
    return x + y

In [ ]:
%%time
# This takes three seconds to run because we call each
# function sequentially, one after the other

x = inc(1)
y = inc(2)
z = add(x, y)

### Parallelize with the <i><small>dask.delayed</small></i> decorator

In [ ]:
from dask import delayed
from dask.distributed import Client

client = Client("172.24.201.182:8786")

distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError


In [ ]:
%%time
# This runs immediately, all it does is build a graph

x = delayed(inc)(1)
y = delayed(inc)(2)
z = delayed(add)(x, y)

### Compute the real value

In [ ]:
%%time
# This actually runs our computation using a local thread pool

z.compute()

### Type Delayed

In [ ]:
z

### View the exec plan

In [ ]:
# Look at the task graph for `z`
z.visualize()

### Parallelize a for loop

In [ ]:
# Data 
data = [1, 2, 3, 4, 5, 6, 7, 8]

In [ ]:
%%time
# Sequential code

results = []
for x in data:
    y = inc(x)
    results.append(y)

total = sum(results)

In [ ]:
# Get Total value
total

In [ ]:
%%time
# Parallel code
results = []

for x in data:
    y = delayed(inc)(x)
    results.append(y)

total = delayed(sum)(results)
print("Before computing:", total)  # Let's see what type of thing total is
result = total.compute()
print("After computing :", result)  # After it's computed

In [ ]:
# Delayed 
total

In [ ]:
# Look at the task graph for `total`
total.visualize()

### Parallelize a for loop with control flow

In [ ]:
def double(x):
    sleep(1)
    return 2 * x

def is_even(x):
    return not x % 2

data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [ ]:
%%time
# Sequential code

results = []
for x in data:
    if is_even(x):
        y = double(x)
    else:
        y = inc(x)
    results.append(y)

total = sum(results)
print(total)

In [ ]:
%%time
results = []
for x in data:
    if is_even(x):  # even
        y = delayed(double)(x)
    else:          # odd
        y = delayed(inc)(x)
    results.append(y)

total = delayed(sum)(results)

In [ ]:
%time total.compute()

In [ ]:
total.visualize()